In [ ]:
%run -i Scraping.py

In [8]:
finalInmig2015.head(5)

,Países,Inmigrantes hombres,Inmigrantes mujeres,Inmigrantes,% Inmigrantes,Var.
0,España,2.884.000,3.007.208,5.891.208,"12,69%","-0,77"
1,Alemania,5.037.961,5.182.457,10.220.418,"12,44%","0,21"
2,Reino Unido,4.023.771,4.387.798,8.411.569,"12,87%","1,57"
3,Francia,3.806.413,4.067.759,7.874.172,"11,82%","0,57"
4,Italia,2.637.798,3.167.530,5.805.328,"9,57%","-0,18"
